# Keras Tuner

## Environment Setup

In [ ]:
import pandas as pd
import tensorflow as tf
import os
import shutil
import numpy as np
from tensorflow import keras
from tensorflow.keras import optimizers
from keras import layers
from keras import models
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
CLASSES = ['Paul_Gauguin',
 'Edgar_Degas',
 'Vincent_van_Gogh',
 'Albrecht_Dürer',
 'Pablo_Picasso',
 'Titian',
 'Francisco_Goya',
 'Marc_Chagall',
 'Pierre-Auguste_Renoir',
 'Alfred_Sisley',
 'Rembrandt']

TRAIN_DIR = '/content/drive/Shareddrives/DeepLearning-ArtClassification/Data/Split-dataset/train'
TEST_DIR = '/content/drive/Shareddrives/DeepLearning-ArtClassification/Data/Split-dataset/test'
VAL_DIR = '/content/drive/Shareddrives/DeepLearning-ArtClassification/Data/Split-dataset/val'

IMAGE_HEIGHT = 224
IMAGE_WIDTH = 224
BATCH_SIZE = 32
RANDOM_SEED = 47
NUM_EPOCHS = 20
PATIENCE = 3

BEST_MODEL_PATH = "/content/drive/Shareddrives/DeepLearning-ArtClassification/Models/Keras_Tuner"

if not os.path.exists(BEST_MODEL_PATH):
    os.makedirs(BEST_MODEL_PATH)

LOCAL_PATH = "/content/model"

if not os.path.exists(LOCAL_PATH):
    os.makedirs(LOCAL_PATH)

###  Compute class weights

In [ ]:
GENERAL_PATH="/content/drive/Shareddrives/DeepLearning-ArtClassification/Data/Split-dataset/train"

class_cardinality={}
for it in os.scandir(GENERAL_PATH):
    if it.is_dir():
      class_cardinality[it.name]=len(os.listdir(it.path))

df = pd.DataFrame(class_cardinality.items(), columns=['Name', 'Num_Paintings'])
df["Class_Weight"] = df.Num_Paintings.sum() / (len(CLASSES)*df.Num_Paintings)
CLASS_WEIGHTS = df['Class_Weight'].to_dict()
CLASS_WEIGHTS

{0: 1.258483452031839,
 1: 0.556193297537493,
 2: 0.44549903603737206,
 3: 1.1925367209210005,
 4: 0.889546935149541,
 5: 1.5342185903983656,
 6: 1.3452754142409316,
 7: 1.6352749047359825,
 8: 1.1620889748549323,
 9: 1.508789552988448,
 10: 1.4923000496770988}

### Dataset Loading

In [ ]:
training_images = tf.keras.preprocessing.image_dataset_from_directory(
    TRAIN_DIR,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    image_size=(IMAGE_HEIGHT,IMAGE_WIDTH),
    interpolation='bilinear',
    shuffle=True,
    seed=RANDOM_SEED
)

test_images = tf.keras.preprocessing.image_dataset_from_directory(
    TEST_DIR,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    image_size=(IMAGE_HEIGHT,IMAGE_WIDTH),
    interpolation='bilinear',
    shuffle=False,
    seed=RANDOM_SEED
)


val_images = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DIR,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    color_mode='rgb',
    image_size=(IMAGE_HEIGHT,IMAGE_WIDTH),
    interpolation='bilinear',
    shuffle=True,
    seed=RANDOM_SEED
)

Found 3004 files belonging to 11 classes.
Found 656 files belonging to 11 classes.
Found 639 files belonging to 11 classes.


## Keras Tuner 

### Building Tuner Model

In [ ]:
! pip install -q -U keras-tuner

import keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 KB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.9 MB/s eta 0:00:00


In [ ]:
def build_model(hp):
  
  # set the hyperparameters
  activation_hp = hp.Choice('activation_function', values=['relu', 'elu', 'gelu'])
  n_neurons_hp = hp.Int('FC_layer', min_value=0, max_value=256, step=16)
  dropout_rate_hp = hp.Float('dropout_rate', min_value = 0, max_value = 0.5)
  lr_hp = hp.Float('learning_rate', min_value=5e-5, max_value=5e-3)

  #definition of the parametric model
  inputs = keras.Input(shape=(224, 224, 3))
  x = layers.Rescaling(1. / 255)(inputs)
  x = layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(7,7), padding="same", activation=activation_hp)(x)
  x = layers.MaxPooling2D(pool_size=(4, 4), strides=4)(x)
  x = layers.Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation=activation_hp)(x)
  x = layers.MaxPooling2D(pool_size=(2, 2))(x)
  x = layers.Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation=activation_hp)(x)
  x = layers.MaxPooling2D(pool_size=(2, 2))(x)
  x = layers.Flatten()(x)
  x = layers.Dense(n_neurons_hp)(x)
  x = layers.Dropout(dropout_rate_hp)(x)
  outputs = layers.Dense(len(CLASSES), activation='softmax', name='predictions')(x)
  model = tf.keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=lr_hp), loss='categorical_crossentropy', metrics=['accuracy'])
  return model


In [ ]:
stop_early=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)]

In [ ]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=30,
    executions_per_trial=1,
    overwrite=True,
    directory="/content/drive/Shareddrives/DeepLearning-ArtClassification/Models/Keras_Tuner",
    project_name="tuning",
)

tuner.search_space_summary()

Search space summary
Default search space size: 4
activation_function (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'elu', 'gelu'], 'ordered': False}
FC_layer (Int)
{'default': None, 'conditions': [], 'min_value': 0, 'max_value': 256, 'step': 16, 'sampling': 'linear'}
dropout_rate (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': None, 'sampling': 'linear'}
learning_rate (Float)
{'default': 5e-05, 'conditions': [], 'min_value': 5e-05, 'max_value': 0.005, 'step': None, 'sampling': 'linear'}


In [ ]:
from sklearn.utils import class_weight

tuner.search(training_images, epochs=20, validation_data=val_images, callbacks=stop_early, class_weight=CLASS_WEIGHTS)

Trial 30 Complete [00h 01m 20s]
val_accuracy: 0.6103286147117615

Best val_accuracy So Far: 0.6525821685791016
Total elapsed time: 01h 19m 13s


### Best Combination Results

In [ ]:
tuner.results_summary()

Results summary
Results in /content/drive/Shareddrives/DeepLearning-ArtClassification/Models/Keras_Tuner2/tuning
Showing 10 best trials
Trial summary
Hyperparameters:
activation_function: relu
FC_layer: 112
dropout_rate: 0.28152827249052687
learning_rate: 0.0011311982340156236
Score: 0.6525821685791016
Trial summary
Hyperparameters:
activation_function: relu
FC_layer: 80
dropout_rate: 0.2755088495585481
learning_rate: 0.0007865509136633198
Score: 0.6510171890258789
Trial summary
Hyperparameters:
activation_function: elu
FC_layer: 16
dropout_rate: 0.2742520352815978
learning_rate: 0.0011016830744080951
Score: 0.6478873491287231
Trial summary
Hyperparameters:
activation_function: relu
FC_layer: 96
dropout_rate: 0.28897281775752265
learning_rate: 0.0018297868909015315
Score: 0.6463223695755005
Trial summary
Hyperparameters:
activation_function: gelu
FC_layer: 16
dropout_rate: 0.11384553571840661
learning_rate: 0.0015536758885320169
Score: 0.6431924700737
Trial summary
Hyperparameters:
act

In [ ]:
# Get the top models.
best_model = tuner.get_best_models(num_models = 1)[0]
best_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 rescaling (Rescaling)       (None, 224, 224, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 8, 8, 32)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 64)          18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 64)         0         
 2D)                                                         

In [ ]:
tf.keras.models.save_model(best_model, os.path.join(BEST_MODEL_PATH, 'kerasTuner_Model.h5'))

In [ ]:
best_model.evaluate(test_images)

21/21 [==============================] - 1s 48ms/step - loss: 1.1612 - accuracy: 0.6189


[1.1612458229064941, 0.6189024448394775]